In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
cd /kaggle/working/

/kaggle/working


In [3]:
!rm -rf reid-custom

In [4]:
!git clone https://github.com/HuangYin0514/reid-custom.git

Cloning into 'reid-custom'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 158 (delta 45), reused 150 (delta 37), pack-reused 0
Receiving objects: 100% (158/158), 582.39 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [5]:
!ls

__notebook__.ipynb  reid-custom


In [6]:
cd  reid-custom

/kaggle/working/reid-custom


In [7]:
!pip install -e ./

Obtaining file:///kaggle/working/reid-custom
  Running setup.py develop for torchreid


In [8]:
ls /kaggle/input/market1501/

Market-1501-v15.09.15/


In [9]:

import torchreid
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

state_dict = {
    'name': 'pcb_p6',
    'pretrain': False,
    'load_path_url': '/home/hy/vscode/reid-custom/log/tensorboard/log/resnet50/model/model.pth.tar-60',
}


datamanager = torchreid.data.ImageDataManager(
    root='/kaggle/input/',
    sources='market1501',
    targets='market1501',
    height=384,
    width=128,
    batch_size_train=64,
    batch_size_test=64,
    transforms=['random_flip', 'random_crop']
)

model = torchreid.models.build_model(
    name=state_dict['name'],
    num_classes=datamanager.num_train_pids,
    loss='softmax',
    pretrained=True
)

if state_dict['pretrain']:
    torchreid.utils.Load_trained_parameters(
        load_path_url=state_dict['load_path_url'],
        device=device
    ).load_trained_model_weights(model)

model = model.to(device)


optimizer = torchreid.optim.build_optimizer(
    model, optim='sgd', lr=0.1, staged_lr=True,
    new_layers=['parts_avgpool', 'dropout', 'conv5', 'classifier'], base_lr_mult=0.01,
    weight_decay=0.0005, momentum=0.9
)


scheduler = torchreid.optim.build_lr_scheduler(
   optimizer, lr_scheduler='multi_step', stepsize=[41,]
)

engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)


engine.run(
    save_dir='log/'+state_dict['name'],
    max_epoch=60,
    eval_freq=30,
    print_freq=20,
    test_only=False
)

/kaggle/working/reid-custom/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


Building train transforms ...
+ resize to 384x128
+ random flip
+ random crop (enlarge to 432x144 and crop 384x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 384x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------
=> Loading test (target) dataset
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gal

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth



=> Start training
epoch: [1/60][20/202]	time 0.455 (0.688)	data 0.000 (0.177)	eta 2:18:42	loss 6.5840 (6.6199)	acc 0.0000 (0.0000)	lr 0.001000
epoch: [1/60][40/202]	time 0.463 (0.572)	data 0.000 (0.088)	eta 1:55:14	loss 6.6235 (6.6044)	acc 1.5625 (0.3125)	lr 0.001000
epoch: [1/60][60/202]	time 0.456 (0.534)	data 0.000 (0.059)	eta 1:47:15	loss 6.5569 (6.5875)	acc 0.0000 (0.5729)	lr 0.001000
epoch: [1/60][80/202]	time 0.466 (0.514)	data 0.000 (0.044)	eta 1:43:13	loss 6.5555 (6.5720)	acc 0.0000 (0.5273)	lr 0.001000
epoch: [1/60][100/202]	time 0.454 (0.503)	data 0.000 (0.036)	eta 1:40:41	loss 6.5745 (6.5621)	acc 3.1250 (0.5312)	lr 0.001000
epoch: [1/60][120/202]	time 0.455 (0.496)	data 0.000 (0.030)	eta 1:39:06	loss 6.4635 (6.5564)	acc 3.1250 (0.5469)	lr 0.001000
epoch: [1/60][140/202]	time 0.455 (0.491)	data 0.000 (0.025)	eta 1:37:59	loss 6.4864 (6.5435)	acc 0.0000 (0.6027)	lr 0.001000
epoch: [1/60][160/202]	time 0.455 (0.487)	data 0.000 (0.022)	eta 1:37:01	loss 6.4237 (6.5350)	acc 0.000

In [10]:
! tar zcf ../logs.tar ./log/

In [11]:
cd /kaggle/working/

/kaggle/working


In [12]:
!rm -rf reid*